Meta Learning

NumerAi

Tanmay Singh
2021569
CSAI
Class of '25

Importing the Dependencies

In [ ]:
!pip install scipy
!pip install numpy
!pip install pandas
!pip install xgboost
!pip install seaborn
!pip install pyarrow
!pip install numerapi
!pip install imblearn
!pip install catboost
!pip install lightgbm
!pip install matplotlib
!pip install cloudpickle
!pip install mplcyberpunk
!pip install scikit-learn
!pip install torchsummary

In [ ]:
import os
import gc
import time
import json
import pickle
import numpy as np
import pandas as pd
import seaborn as sb
import xgboost as xgb
import lightgbm as lgb
import cloudpickle as cp
import matplotlib.pyplot as plt

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [ ]:
from torchsummary import summary
from torch.utils.data import DataLoader, Dataset

In [ ]:
from tqdm import tqdm
from scipy import stats
from numerapi import NumerAPI
from scipy.stats import pearsonr
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

In [ ]:
from sklearn.tree import *
from sklearn.metrics import *
from sklearn.ensemble import *
from sklearn.linear_model import *
from sklearn.decomposition import *
from sklearn.preprocessing import *
from sklearn.neural_network import *
from sklearn.model_selection import *
from sklearn.cluster._kmeans import KMeans
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import LocalOutlierFactor
from sklearn.neighbors import KNeighborsClassifier
from imblearn.under_sampling import RandomUnderSampler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

Function to compute NumerAi Correlation

In [ ]:
def label_frequency(predictions):
    unique, counts = np.unique(predictions, return_counts=True)
    label_frequencies = dict(zip(unique, counts))
    print("Label frequencies:", label_frequencies)

Loading the Dataset

In [8]:
NumerAi = NumerAPI()

In [9]:
all_datasets = NumerAi.list_datasets()
dataset_versions = list(set(d.split('/')[0] for d in all_datasets))
print("Available versions:\n", dataset_versions)

Available versions:
 ['v5.0']


In [10]:
DATA_VERSION = "v5.0"

current_version_files = [f for f in all_datasets if f.startswith(DATA_VERSION)]
print("availbable", DATA_VERSION, "files:\n", current_version_files)

availbable v5.0 files:
 ['v5.0/features.json', 'v5.0/live.parquet', 'v5.0/live_benchmark_models.parquet', 'v5.0/live_example_preds.csv', 'v5.0/live_example_preds.parquet', 'v5.0/meta_model.parquet', 'v5.0/train.parquet', 'v5.0/train_benchmark_models.parquet', 'v5.0/validation.parquet', 'v5.0/validation_benchmark_models.parquet', 'v5.0/validation_example_preds.csv', 'v5.0/validation_example_preds.parquet']


Creating a Feature Set

In [11]:
NumerAi.download_dataset(f"{DATA_VERSION}/features.json")

feature_metadata = json.load(open(f"{DATA_VERSION}/features.json"))
for metadata in feature_metadata:
  print(metadata, len(feature_metadata[metadata]))

2024-11-06 15:23:55,664 INFO numerapi.utils: target file already exists
2024-11-06 15:23:55,665 INFO numerapi.utils: download complete


feature_sets 17
targets 37


In [ ]:
feature_sets = feature_metadata["feature_sets"]
for feature_set in ["small", "medium", "all"]:
  print(feature_set, len(feature_sets[feature_set]))

In [ ]:
feature_sets = feature_metadata["feature_sets"]
feature_sets.keys()

In [ ]:
for feature_set in feature_sets:
  print(f'Feature Set: {feature_set:<25}', f'Size: {len(feature_sets[feature_set])}')

Loading the Saved Experts & the Meta-Model

In [14]:
with open('numerai_expert1.pkl', 'rb') as f:
    expert1 = pickle.load(f)
print("Model loaded successfully!")

Model loaded successfully!


In [15]:
with open('numerai_expert2.pkl', 'rb') as f:
    expert2 = pickle.load(f)
print("Model loaded successfully!")

Model loaded successfully!


In [16]:
with open('numerai_expert3.pkl', 'rb') as f:
    expert3 = pickle.load(f)
print("Model loaded successfully!")

Model loaded successfully!


In [17]:
with open('numerai_expert4.pkl', 'rb') as f:
    expert4 = pickle.load(f)
print("Model loaded successfully!")

Model loaded successfully!


In [18]:
with open('numerai_expert5.pkl', 'rb') as f:
    expert5 = pickle.load(f)
print("Model loaded successfully!")

Model loaded successfully!


In [19]:
with open('numerai_expert6.pkl', 'rb') as f:
    expert6 = pickle.load(f)
print("Model loaded successfully!")

Model loaded successfully!


In [20]:
with open('numerai_meta_model.pkl', 'rb') as f:
    meta_model = pickle.load(f)
print("Meta Model loaded successfully!")

Meta Model loaded successfully!


Loading the Live Data Set, with a 'medium' feature set

In [24]:
NumerAi.download_dataset(f"{DATA_VERSION}/live.parquet")
feature_set = feature_sets["medium"]

live_data = pd.read_parquet(f"{DATA_VERSION}/live.parquet")
features = [f for f in live_data.columns if "feature" in f]
live = live_data[["era"] + feature_set]

2024-11-06 15:24:02,971 INFO numerapi.utils: target file already exists
2024-11-06 15:24:02,973 INFO numerapi.utils: starting download
v5.0/live.parquet: 15.6MB [00:00, 23.5MB/s]                   


Preprocessing the Live Set (in the same manner as the Training & Validation Sets)

In [25]:
live.rename(columns=lambda x: f'feature {feature_set.index(x)}' if x in feature_set else x, inplace=True)

In [26]:
live

,era,feature 0,feature 1,feature 2,feature 3,feature 4,feature 5,feature 6,feature 7,feature 8,...,feature 695,feature 696,feature 697,feature 698,feature 699,feature 700,feature 701,feature 702,feature 703,feature 704
id,,,,,,,,,,,,,,,,,,,,,
n0039880040fb023,X,4,0,3,0,2,0,1,1,0,...,3,1,3,3,1,3,2,4,0,4
n005b7bee7ec0816,X,3,1,4,3,2,2,0,0,3,...,3,1,2,4,4,1,1,2,1,1
n00693f63f9be56b,X,2,3,2,0,3,2,2,0,2,...,0,3,2,2,3,4,2,2,3,2
n00b8288605a5c6d,X,2,3,0,2,0,0,4,2,1,...,2,1,3,2,1,4,2,0,3,0
n00b97850c393025,X,4,4,2,4,3,2,1,4,3,...,2,3,2,4,2,1,0,4,3,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
nffceeac42d236e9,X,0,0,1,0,3,4,3,1,0,...,3,4,2,2,2,2,0,2,1,2
nffcf2396cb13040,X,4,4,0,4,3,1,3,3,4,...,3,4,3,1,4,1,0,4,4,2
nffcfec6bec2fa94,X,1,0,1,3,3,4,3,3,0,...,3,4,2,2,1,2,0,2,0,2


Setting a Numeric Value to the 'era' column

In [27]:
live['era'] = 1

In [28]:
live

,era,feature 0,feature 1,feature 2,feature 3,feature 4,feature 5,feature 6,feature 7,feature 8,...,feature 695,feature 696,feature 697,feature 698,feature 699,feature 700,feature 701,feature 702,feature 703,feature 704
id,,,,,,,,,,,,,,,,,,,,,
n0039880040fb023,1,4,0,3,0,2,0,1,1,0,...,3,1,3,3,1,3,2,4,0,4
n005b7bee7ec0816,1,3,1,4,3,2,2,0,0,3,...,3,1,2,4,4,1,1,2,1,1
n00693f63f9be56b,1,2,3,2,0,3,2,2,0,2,...,0,3,2,2,3,4,2,2,3,2
n00b8288605a5c6d,1,2,3,0,2,0,0,4,2,1,...,2,1,3,2,1,4,2,0,3,0
n00b97850c393025,1,4,4,2,4,3,2,1,4,3,...,2,3,2,4,2,1,0,4,3,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
nffceeac42d236e9,1,0,0,1,0,3,4,3,1,0,...,3,4,2,2,2,2,0,2,1,2
nffcf2396cb13040,1,4,4,0,4,3,1,3,3,4,...,3,4,3,1,4,1,0,4,4,2
nffcfec6bec2fa94,1,1,0,1,3,3,4,3,3,0,...,3,4,2,2,1,2,0,2,0,2


Generating Predictions from Experts

EXPERT-1 (XGBOOST CLASSIFIER)

In [29]:
expert1_live_pred = expert1.predict(live)

In [30]:
expert1_live_pred

array([3, 1, 2, ..., 2, 2, 2])

EXPERT-2 (RANDOM FOREST CLASSIFIER)

In [31]:
expert2_live_pred = expert2.predict(live)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.2s


[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    0.3s
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:    0.6s
[Parallel(n_jobs=4)]: Done 1000 out of 1000 | elapsed:    0.8s finished


In [32]:
expert2_live_pred

array([2., 1., 2., ..., 2., 2., 2.], dtype=float32)

EXPERT-3 (ADABOOST CLASSIFIER with DECISION TREE CLASSIFIER as BASE ESTIMATOR)

In [33]:
expert3_live_pred = expert3.predict(live)

In [34]:
expert3_live_pred

array([3., 1., 2., ..., 2., 2., 2.], dtype=float32)

EXPERT-4 (LOGISTIC REGRESSION)

In [35]:
expert4_live_pred = expert4.predict(live)

In [36]:
expert4_live_pred

array([2., 0., 2., ..., 1., 2., 2.], dtype=float32)

EXPERT-5 (CATBOOST CLASSIFIER)

In [37]:
expert5_live_pred = expert5.predict(live)

In [ ]:
if expert5_live_pred.ndim > 1:
    expert5_live_pred = expert5_live_pred.ravel()

In [ ]:
expert5_live_pred

EXPERT-6 (HISTOGRAM-BASED GRADIENT BOOST CLASSIFIER)

In [39]:
expert6_live_pred = expert6.predict(live)

In [40]:
expert6_live_pred

array([2., 4., 2., ..., 2., 2., 2.], dtype=float32)

Generating Predictions from Meta-Model (LIGHTGBM)

In [41]:
meta_live_x = np.column_stack((expert1_live_pred, expert2_live_pred, expert3_live_pred, expert4_live_pred, expert5_live_pred, expert6_live_pred))

In [42]:
meta_live_x

array([[3., 2., 3., 2., 3., 2.],
       [1., 1., 1., 0., 3., 4.],
       [2., 2., 2., 2., 2., 2.],
       ...,
       [2., 2., 2., 1., 3., 2.],
       [2., 2., 2., 2., 2., 2.],
       [2., 2., 2., 2., 2., 2.]])

In [43]:
meta_live_predictions = meta_model.predict(meta_live_x)

In [44]:
meta_live_predictions

array([2.40410961, 1.45875632, 2.07233549, ..., 2.19886853, 2.07233549,
       2.07233549])

In [45]:
bins = [0.5, 1.5, 2.5, 3.5]

live_predictions = np.digitize(meta_live_predictions, bins)

In [46]:
live_predictions

array([2, 1, 2, ..., 2, 2, 2])

In [47]:
label_frequency(live_predictions)

Label frequencies: {1: 1152, 2: 3773, 3: 1306}


Converting the Classes back into Numeric Form for Submission to NumerAi Platform

Creating a Label Map (Both forward & reverse Mapping)

In [48]:
labels_dict = {0.0: 0, 0.25: 1, 0.5: 2, 0.75: 3, 1.0: 4}
reverse_labels_dict = {v: k for k, v in labels_dict.items()}
og_live_predictions = np.vectorize(reverse_labels_dict.get)(live_predictions)

In [49]:
og_live_predictions

array([0.5 , 0.25, 0.5 , ..., 0.5 , 0.5 , 0.5 ])

In [50]:
label_frequency(og_live_predictions)

Label frequencies: {0.25: 1152, 0.5: 3773, 0.75: 1306}


Preparing Data into Submission Format

In [51]:
submission = pd.Series(og_live_predictions, index=live.index).to_frame("prediction")

submission

,prediction
id,
n0039880040fb023,0.50
n005b7bee7ec0816,0.25
n00693f63f9be56b,0.50
n00b8288605a5c6d,0.25
n00b97850c393025,0.50
...,...
nffceeac42d236e9,0.50
nffcf2396cb13040,0.50
nffcfec6bec2fa94,0.50


Saving the Live Predictions into Pickle & CSV formats

In [52]:
p = cloudpickle.dumps(submission)

with open("live_submission.pkl", "wb") as f:
    f.write(p)

print("Predictions saved to 'live_submission.pkl' using cloudpickle.")

Predictions saved to 'live_submission.pkl' using cloudpickle.


In [53]:
submission = submission.reset_index()
submission = submission.rename(columns={'index': 'id'})
submission.to_csv("live_submission.csv", index=False)

print("Predictions saved to 'live_submission.csv' with ID and Prediction columns.")

Predictions saved to 'live_submission.csv' with ID and Prediction columns.
